In [1]:
import os
#from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
#from langchain.chat_models import AzureChatOpenAI
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()


True

In [2]:
def ask_question(question):
    # Initialize the Azure ChatGPT model
    model = AzureOpenAI(
        azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),  # Replace with your Azure deployment name
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT") ,  # Replace with your Azure OpenAI endpoint
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    )
    # Use the chat.completions.create method instead
    response = model.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": question}
        ]
    )
    return response.choices[0].message.content

In [3]:
if __name__ == "__main__":
    # Test question
    question = "Who is the Prime Minister of Russia?"
    answer = ask_question(question)
    print("Answer:", answer)


Answer: As of September 2021, the Prime Minister of Russia is Mikhail Mishustin. He assumed office on January 16, 2020, following the resignation of Dmitry Medvedev.


In [ ]:
EndPoint = os.getenv("AZURE_OPENAI_ENDPOINT")
print(EndPoint)
Vers = os.getenv("AZURE_OPENAI_API_VERSION")
print(Vers)

In [ ]:
embeddings_endpoint = os.getenv("AZURE_OPENAI_EMBEDDINGS_ENDPOINT")
embeddings_key = os.getenv("AZURE_OPENAI_EMBEDDINGS_KEY")
embeddings_model = os.getenv("AZURE_OPENAI_EMBEDDINGS_MODEL")


In [ ]:
client = AzureOpenAI(
    azure_endpoint= EndPoint,
    api_key= os.getenv("AZURE_OPENAI_API_KEY"),
    api_version= Vers
)


In [ ]:
# Create chat completion
response = client.chat.completions.create(
    model="gpt-35-turbo-16k",  # Your deployed model name
    messages=[
        {"role": "user", "content": "Who is Prime Minister of India?"}
    ]
)

# Print the response
print(response.choices[0].message.content)